# sacar para dataset

In [6]:
import arcpy
import os

# --- Rutas base
tiles_shp = r"C:\inteia\output\tiles_para_etiquetar_AGRUPADO.shp"
gdb_path = r"C:\Users\diana\Documents\ArcGIS\Projects\INTEIA\INTEIA.gdb"
vector_clasificacion = os.path.join(gdb_path, "clasificacion_vect_Intersect_TILES")
image_raster = r"C:\inteia\output\cropped_image_1_subset.tif"  # nombre actualizado

# --- Crear carpetas de salida
output_root = r"C:\inteia\output\dataset"
images_folder = os.path.join(output_root, "images")
labels_folder = os.path.join(output_root, "labels")

os.makedirs(images_folder, exist_ok=True)
os.makedirs(labels_folder, exist_ok=True)

# --- Capa temporal de tiles
arcpy.MakeFeatureLayer_management(tiles_shp, "tiles_layer")

# --- Iterar sobre cada tile
with arcpy.da.SearchCursor(tiles_shp, ["tile_id", "SHAPE@"]) as cursor:
    for tile_id, geometry in cursor:
        print(f"Procesando tile {tile_id}...")

        # 1. Exportar imagen en .tif
        out_image = os.path.join(images_folder, f"{int(tile_id)}.tif")
        arcpy.Clip_management(
            in_raster=image_raster,
            rectangle="",  # Usa geometría
            out_raster=out_image,
            in_template_dataset=geometry,
            nodata_value="0",
            clipping_geometry="ClippingGeometry",
            maintain_clipping_extent="MAINTAIN_EXTENT"
        )

        # 2. Exportar geojson de etiquetas
        label_output = os.path.join(labels_folder, f"{int(tile_id)}.geojson")
        where_clause = f"tile_id = {tile_id}"
        arcpy.MakeFeatureLayer_management(vector_clasificacion, "class_layer", where_clause)
        arcpy.conversion.FeaturesToJSON(
            in_features="class_layer",
            out_json_file=label_output,
            geoJSON="GEOJSON"
        )

print("✅ ¡Proceso completado! GeoTIFFs y GeoJSONs generados por tile.")


Procesando tile 6.0...
Procesando tile 7.0...
Procesando tile 18.0...
Procesando tile 22.0...
Procesando tile 23.0...
Procesando tile 26.0...
Procesando tile 30.0...
Procesando tile 32.0...
Procesando tile 34.0...
Procesando tile 39.0...
✅ ¡Proceso completado! GeoTIFFs y GeoJSONs generados por tile.
